# CYCLOTOMIC FACTORS OF POLYNOMIALS
A <span style="color: navy;">**SageMath**</span> tutorial<br>
<span style="color: maroon;">**VIASM**</span> REU 2025

## Work environment 📚
We start by creating a polynomial ring named ``R`` with a single variable, ``x``. The coefficients of the polynomials in this ring are integers, which is represented by ``ZZ``.

In [2]:
R.<x> = PolynomialRing(ZZ)

Then we can define a polynomial using a vector (list) of its coefficients, starting from $x^0$ (going backwards on the actual polynomial). For a polynomial of degree $n$, this vector is in the form:<br>
``v = [x0_coeff, x1_coeff, x2_coeff, ..., xn_coeff]``<br>
For example, the polynomial $x^2 - 5x + 6$ will have the coefficients vector ``v = [6, -5, 1]``. The polynomial will then be declared using the ring ``R`` by running:<br>
``Fx = R(v)``

In [3]:
v = [6, -5, 1]
Fx = R(v)
Fx

x^2 - 5*x + 6

## Some important functions ⚙️

### Coprime check
Two integers are considered coprime (or relatively prime) if their greatest common divisor (gcd) is 1.<br>
The function returns 1 (``True``) if ``m``,``n``, are coprime, and 0(``False``) otherwise.

In [4]:
def is_coprime(m,n):
    if gcd(m,n) == 1:
        return 1
    else:
        return 0

### Square-free check
An integer is considered square-free if, in its prime factorization, every prime factor appears with an exponent of exactly 1. In other words, if integer $n$ has a prime factor $p$ with exponent $\geq2$, $p^2|n$.<br>
To check for this, we use the built-in function prime factorization function ``factor(n)``. For example:

In [5]:
factor(12)

2^2 * 3

This secretly wraps a list of pairs (prime, exponent), and you can get this list by running the ``list()`` function.

In [6]:
list(factor(12))

[(2, 2), (3, 1)]

For our purposes, we will iterate through each ``pair`` in the list, check if any exponent (the second component of the ``pair``, hence ``pair[1]``) is $\geq2$.

In [7]:
def not_square_free(n):
    for pair in factor(n):
        if pair[1] >= 2:
            return True
    return False

### Möbius function
The Möbius function, $\mu(n)$, is defined as:<br>
$
\mu(n) =
\begin{cases}
0 & \text{if } n \text{ is not square-free} \\
1 & \text{if } n=1\\
(-1)^k & \text{if } n \text{ is a product of } k \text{ distinct primes}
\end{cases}
$<br>
The n=1 case is a simple if-then. The $n$ is not square-free case can be checked using the previously defined ``not_square_free(n)``.<br>
For the $n$ is a product of $k$ distinct primes case, we count the number of distinct prime factors using the length, ``len()``, of the previously mentioned ``list(factor(n))``. For example:

In [8]:
list(factor(30))

[(2, 1), (3, 1), (5, 1)]

In [9]:
len(list(factor(30)))

3

The Möbius function, $\mu(n)$, is then defined as:

In [10]:
def mobius(n):
    if n == 1:
        return 1
    elif not_square_free(n):
        return 0
    else:
        return (-1)**len(list(factor(n)))

### Fekete polynomials
A Fekete polynomial, $F_n(x)$ is defined as:<br>
$F_n(x) = \sum_{\substack{1 \leq a \leq n \\ \text{gcd}(a,n)=1}} x^a$<br>
Since $a$ starts at 1, all Fekete polynomials have no $x^0$. Hence, the coefficients vector ``v`` starts with ``[0]``.<br>
We then iterate through every $a$ from 1 to $n$. If gcd($a,n$)=1, the coefficient for $x^a$ will be 1. Otherwise, coefficient for $x^a$ will be 0. This is done by using the previously defined ``is_coprime(a,n)`` function.<br>
With the complete coefficient vector ``v``, the Fekete polynomial is simply ``R(v)``.

In [11]:
def fekete(n):
    v = [0]
    v += [is_coprime(a,n) for a in range (1,n+1)]
    Fn_x = R(v)
    return Fn_x

For example, $F_{10}(x) = x^9 + x^7 + x^3 + x$.

In [12]:
fekete(10)

x^9 + x^7 + x^3 + x

### Necklace polynomials
A necklace polynomial, $M_n(x)$ is defined as:<br>
$M_n(x) = \frac{1}{n} \sum_{a|n} \mu(a) x^{\frac{n}{a}}$<br>
Since x's are raised to $\frac{n}{a}$ power, we take a different approach of constructing the coefficients vector ``v`` as to Fekete polynomials.<br>
Note that for all $n$, $M_n(x)$ includes $x$ and $x^n$. Hence, we start by creating a vector of $(n+1)$ 0s as place holder for coefficients of $x^0$ to $x^n$. This is done by declaring ``v = [0]*(n+1)``.<br>
Since $n \nmid 0$, we iterate through all $a$ from 1 to $n$ and check for divisibility using ``if n%a == 0``. If $n \nmid a$, we change the $\frac{n}{a}$ element of ``v`` to ``mobius(a)``.<br>
Finally, we multiply our polynomial with $\frac{1}{n}$ and return the complete neckplace polynomial.

In [13]:
def necklace(n):
    v = [0]*(n+1)
    for a in range (1, n+1):
        if n%a == 0:
            v[n//a] = mobius(a)           
    Mn_x = R(v)*(1/n)
    return Mn_x

For example, $M_{15}(x)=\frac{1}{15}(x^{15}-x^5-x^3+x)$.

In [14]:
necklace(15)

1/15*x^15 - 1/15*x^5 - 1/15*x^3 + 1/15*x

### Polynomial factorization
Polynomial factorization can be done by using the polynomial class method ``.factor()``. See examples below:

$F_{10}(x) = x^9 + x^7 + x^3 + x = x(x^2+1)^2(x^4-x^2+1)$.

In [15]:
F_10 = fekete(10)
F_10.factor()

x * (x^2 + 1)^2 * (x^4 - x^2 + 1)

$M_{15}(x)=\frac{1}{15}(x^{15}-x^5-x^3+x)=\frac{1}{15}x(x-1)(x+1)(x^2+1)(x^{10}+x^6+x^2-1)$

In [16]:
M_15 = necklace(15)
M_15.factor()

(1/15) * (x - 1) * x * (x + 1) * (x^2 + 1) * (x^10 + x^6 + x^2 - 1)

Similar to the integer prime factorization function, this also wraps a list of pairs (factor, exponent), and you can get this list by running the list() function.

In [17]:
list(F_10.factor())

[(x, 1), (x^2 + 1, 2), (x^4 - x^2 + 1, 1)]

### Cyclotomic factors

The polynomial class method ``.is_cyclotomic()`` returns ``True`` if a polynomial is cyclotomic, ``False`` if otherwise.<br>
For example, $\Phi_3(x) = x^2+x+1$.

In [18]:
phi3 = R([1, 1, 1])
phi3.is_cyclotomic()

True

Adding the argument ``certificate=True`` to the method returns the degree of the cyclotomic factor.

In [19]:
phi3.is_cyclotomic(certificate=True)

3

The polynomial class ``.cyclotomic_part()`` returns the part of the polynomial consisting of cyclotomic factors. Note that this has not been factored down to irreducible components. For example:

In [20]:
F_10.cyclotomic_part()

x^8 + x^6 + x^2 + 1

We can define a function ``cyclotomic_factor(f)`` that takes a polynomial ``f`` and return a list of its cyclotomic factors as follows:
* Retrieve the cyclotomic part of the ``f`` using ``f_cyclotomic = f.cyclotomic_part()``.
* Factor ``f_cyclotomic`` down to irreducible components using ``f_cyclotomic.factor()``.
* Recall that each cyclotomic factor in this list is a pair (factor, exponent).
  * The degree of the cyclotomic factor can be retrieved by ``factor[0].is_cyclotomic(certificate=True)``.
  * The exponent of the cyclotomic factor is simply the second element in the pair, hence ``factor[1]``.
* Return a list of cyclotomic factor degrees, each repeated by a number of times depending on their exponent.

In [21]:
def cyclotomic_factor(f):
    f_cyclotomic = f.cyclotomic_part()
    phi_list = []
    for factor in f_cyclotomic.factor():
        degree = factor[0].is_cyclotomic(certificate=True)
        exponent = factor[1]
        phi_list += [degree for _ in range(exponent)]
    return phi_list

$F_{10}(x) = x^9 + x^7 + x^3 + x = x(x^2+1)^2(x^4-x^2+1)=x \Phi_4(x)^2 \Phi_{12}(x)$.

In [22]:
cyclotomic_factor(fekete(10))

[4, 4, 12]

As for any irreducible component of a polynomial that is not cyclotomic, we can define a function ``non_cyclotomic_factor(f)`` that takes a polynomial ``f`` and return a list of the non-cyclotomic factors as follows:
* Factor ``f`` using ``f.factor()``.
* Test each factor, which is the first element of the (factor, exponent) pair, hence ``pair[0]``, against the ``is_cyclotomic()`` test.
* Return a list of any factor that fails the ``is_cyclotomic()`` test.

In [23]:
def non_cyclotomic_factor(f):
    remanent = []
    for pair in f.factor():
        factor = pair[0]
        if not factor.is_cyclotomic():
            remanent.append(factor)
    return remanent

$M_{15}(x)=\frac{1}{15}(x^{15}-x^5-x^3+x)=\frac{1}{15}x(x-1)(x+1)(x^2+1)(x^{10}+x^6+x^2-1)$<br>
$M_{15}(x)=\frac{1}{15} x \Phi_1(x) \Phi_2(x) \Phi_4(x) g_{15}(x)$<br>
where $g_{15}(x)=x^{10}+x^6+x^2-1$ is irreducible.

In [24]:
non_cyclotomic_factor(necklace(15))

[x, x^10 + x^6 + x^2 - 1]

## Sample data 📊

### Fekete polynomials

In [25]:
for n in range(3,16):
    Fn = fekete(n)
    phi_list = cyclotomic_factor(Fn)
    print(f'F{n}: {phi_list}')

F3: [2]
F4: [4]
F5: [2, 4]
F6: [8]
F7: [2, 6, 3]
F8: [4, 8]
F9: [2, 9]
F10: [4, 4, 12]
F11: [2, 10, 5]
F12: [4, 12, 8]
F13: [2, 6, 4, 3, 12]
F14: [6, 3, 16]
F15: [2, 4, 8]


### Neckplace polynomials

In [26]:
for n in range(2,16):
    Mn = necklace(n)
    phi_list = cyclotomic_factor(Mn)
    print(f'M{n}: {phi_list}')

M2: [1]
M3: [1, 2]
M4: [1, 2]
M5: [1, 2, 4]
M6: [1, 2]
M7: [1, 2, 6, 3]
M8: [1, 2, 4]
M9: [1, 2, 6, 3]
M10: [1, 2, 6, 4]
M11: [1, 2, 10, 5]
M12: [1, 2, 4]
M13: [1, 2, 6, 4, 3, 12]
M14: [1, 2, 6, 3]
M15: [1, 2, 4]
